In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Function to load images and labels
def load_data(data_dir):
    images = []
    labels = []
    for label, folder in enumerate(['with_mask', 'without_mask']):
        folder_path = os.path.join(data_dir, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Error: Unable to read image '{img_path}'. Skipping...")
                    continue
                img = cv2.resize(img, (100, 100))  # Resize images to a fixed size
                img = img.astype('float32') / 255.0  # Normalize pixel values
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error: Unable to process image '{img_path}': {e}. Skipping...")
                continue
    return np.array(images), np.array(labels)

# Load the dataset
data_dir = "C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset"
images, labels = load_data(data_dir)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define and compile the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Save the trained model
model.save("mask_detection_model.h5")


Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_œ¬‘ÿ.png'. Skipping...
Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png'. Skipping...
Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132400.png'. Skipping...
Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_≈˙◊¢ 2020-02-24 171804.png'. Skipping...
Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_≈˙◊¢ 2020-02-24 172039.png'. Skipping...
Error: Unable to read image 'C:/Users/HP/Downloads/Face-Mask-Detection-master/Face-Mask-Detection-master/dataset\with_mask\0_0_≈˙◊¢ 2020-02-24 202509.png'. Skipping...
Error: Una

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("mask_detection_model.h5")

# Use OpenCV to detect faces and predict mask wearing
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

# Initialize lists to store accuracy and bounding box coordinates
accuracies = []
bounding_boxes = []

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    accuracies = []  # Reset accuracy list for each frame
    bounding_boxes = []  # Reset bounding box list for each frame
    
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (100, 100))  # Resize face to match model input size
        face_roi_normalized = face_roi_resized.astype('float32') / 255.0  # Normalize pixel values
        face_roi_input = np.expand_dims(face_roi_normalized, axis=0)  # Add batch dimension
        prediction = model.predict(face_roi_input)[0][0]
        
        # Store accuracy and bounding box coordinates
        accuracies.append(prediction)
        bounding_boxes.append((x, y, w, h))
    
    # Display bounding boxes, mask predictions, and individual accuracies
    for (x, y, w, h), accuracy in zip(bounding_boxes, accuracies):
        # Draw a green box if the person is wearing a mask, otherwise draw a red box
        color = (0, 255, 0) if accuracy <= 0.5 else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        
        # Display prediction text
        text = "Mask" if accuracy <= 0.5 else "No Mask"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Display individual accuracy
        accuracy_text = f"Accuracy: {100 - accuracy * 100:.2f}%"
        cv2.putText(frame, accuracy_text, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imshow('Mask Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━